In [1]:
import sys
import time
import contextlib
from itertools import cycle
import threading   

@contextlib.contextmanager
def process_runindicator(desc="Process", task_id=None, update_message_callback=None):
    spinner = cycle(["|", "/", "-", "\\"])
    is_running = True
    lock = threading.Lock()

    # Store task information
    if not hasattr(process_runindicator, "tasks"):
        process_runindicator.tasks = {}
    task_id = task_id or id(threading.current_thread())
    process_runindicator.tasks[task_id] = desc

    def spinner_task():
        while is_running:
            with lock:
                sys.stdout.write("\033[H\033[J")  # Clear screen
                for tid, d in process_runindicator.tasks.items():
                    current_desc = d
                    if update_message_callback:
                        current_desc = update_message_callback()
                    sys.stdout.write(f"{current_desc}: {next(spinner)}\n")
                sys.stdout.flush()
            time.sleep(0.1)

    try:
        spinner_thread = threading.Thread(target=spinner_task)
        spinner_thread.start()
        yield
    finally:
        is_running = False
        process_runindicator.tasks.pop(task_id, None)
        spinner_thread.join()
        with lock:
            sys.stdout.write("\033[H\033[J")  # Clear screen
            for tid, d in process_runindicator.tasks.items():
                sys.stdout.write(f"{d}: ✔\n")
            sys.stdout.flush()


if __name__ == "__main__":
    with process_runindicator(desc="Task 1", task_id="task1"), \
         process_runindicator(desc="Task 2", task_id="task2"):
        time.sleep(5)


Task 1: |
Task 1: |
Task 2: /
Task 1: -
Task 2: \
Task 1: /
Task 2: -
Task 1: \
Task 2: |
Task 1: |
Task 2: /
Task 1: /
Task 2: -
Task 1: -
Task 2: \
Task 1: \
Task 2: |
Task 1: |
Task 2: /
Task 1: -
Task 2: \
Task 1: /
Task 2: -
Task 1: |
Task 2: /
Task 1: \
Task 2: |
Task 1: /
Task 2: -
Task 1: -
Task 2: \
Task 1: \
Task 2: |
Task 1: |
Task 2: /
Task 1: /
Task 2: -
Task 1: -
Task 2: \
Task 1: \
Task 2: |
Task 1: |
Task 2: /
Task 1: /
Task 2: -
Task 1: -
Task 2: \
Task 1: \
Task 2: |
Task 1: |
Task 2: /
Task 1: /
Task 2: -
Task 1: -
Task 2: \
Task 1: \
Task 2: |
Task 1: |
Task 2: /
Task 1: /
Task 2: -
Task 1: -
Task 2: \
Task 1: \
Task 2: |
Task 1: |
Task 2: /
Task 1: -
Task 2: \
Task 1: /
Task 2: -
Task 1: |
Task 2: /
Task 1: \
Task 2: |
Task 1: -
Task 2: \
Task 1: /
Task 2: -
Task 1: |
Task 2: /
Task 1: \
Task 2: |
Task 1: -
Task 2: \
Task 1: /
Task 2: -
Task 1: |
Task 2: /
Task 1: \
Task 2: |
Task 1: -
Task 2: \
Task 1: /
Task 2: -
Task 1: |
Task 2: /
Task 1: \
Task 2: |
Task 1: -


In [2]:
import datetime

@contextlib.contextmanager
def process_runindicator(desc="Process", update_message_callback=None):
    spinner = cycle(["|", "/", "-", "\\"])
    is_running = True
    start_time = time.time()

    def spinner_task():
        while is_running:
            elapsed_time = str(datetime.timedelta(seconds=int(time.time() - start_time)))
            current_desc = f"{desc} [{elapsed_time}]"
            if update_message_callback:
                current_desc = update_message_callback()
            sys.stdout.write(f"\r{current_desc}: {next(spinner)}")
            sys.stdout.flush()
            time.sleep(0.1)

    try:
        spinner_thread = threading.Thread(target=spinner_task)
        spinner_thread.start()
        yield
    finally:
        is_running = False
        spinner_thread.join()
        elapsed_time = str(datetime.timedelta(seconds=int(time.time() - start_time)))
        sys.stdout.write(f"\r{desc} [{elapsed_time}]: ✔\n")
        sys.stdout.flush()


if __name__ == "__main__":
    with process_runindicator(desc="Downloading Data"):
        time.sleep(5)


In [3]:
@contextlib.contextmanager
def process_runindicator(desc="Process", update_message_callback=None):
    spinner = cycle(["|", "/", "-", "\\"])
    is_running = True
    start_time = time.time()

    def spinner_task():
        while is_running:
            elapsed_time = str(datetime.timedelta(seconds=int(time.time() - start_time)))
            current_desc = f"{desc} [{elapsed_time}]"
            if update_message_callback:
                current_desc = update_message_callback()
            sys.stdout.write(f"\r{current_desc}: {next(spinner)}")
            sys.stdout.flush()
            time.sleep(0.1)

    try:
        spinner_thread = threading.Thread(target=spinner_task)
        spinner_thread.start()
        yield
    except Exception as e:
        is_running = False
        spinner_thread.join()
        elapsed_time = str(datetime.timedelta(seconds=int(time.time() - start_time)))
        sys.stdout.write(f"\r{desc} [{elapsed_time}]: ✖ Error: {str(e)}\n")
        sys.stdout.flush()
        raise
    else:
        is_running = False
        spinner_thread.join()
        elapsed_time = str(datetime.timedelta(seconds=int(time.time() - start_time)))
        sys.stdout.write(f"\r{desc} [{elapsed_time}]: ✔\n")
        sys.stdout.flush()


if __name__ == "__main__":
    try:
        with process_runindicator(desc="Failing Task"):
            raise ValueError("Simulated error!")  # Simulate an error
    except Exception as e:
        print(f"Handled error: {e}")


Failing Task [0:00:00]: ✖ Error: Simulated error!
Handled error: Simulated error!


In [ ]:
@contextlib.contextmanager
def process_runindicator(desc="Process", task_id=None, parent_id=None, update_message_callback=None):
    spinner = cycle(["|", "/", "-", "\\"])
    is_running = True
    lock = threading.Lock()
    start_time = time.time()

    # Initialize tasks tree if not already present
    if not hasattr(process_runindicator, "tasks"):
        process_runindicator.tasks = {}
        process_runindicator.hierarchy = {}

    # Assign task_id and register the task
    task_id = task_id or id(threading.current_thread())
    process_runindicator.tasks[task_id] = {"desc": desc, "parent_id": parent_id}
    if parent_id:
        process_runindicator.hierarchy.setdefault(parent_id, []).append(task_id)

    def render_spinner():
        # Recursively render the hierarchy
        def render_task(task_id, depth=0):
            task = process_runindicator.tasks[task_id]
            elapsed_time = str(datetime.timedelta(seconds=int(time.time() - start_time)))
            current_desc = f"{task['desc']} [{elapsed_time}]"
            if update_message_callback:
                current_desc = update_message_callback()
            sys.stdout.write(f"{'   ' * depth}{current_desc}: {next(spinner)}\n")
            for child_id in process_runindicator.hierarchy.get(task_id, []):
                render_task(child_id, depth + 1)

        with lock:
            sys.stdout.write("\033[H\033[J")  # Clear the screen
            for root_task_id in process_runindicator.tasks:
                if process_runindicator.tasks[root_task_id]["parent_id"] is None:
                    render_task(root_task_id)
            sys.stdout.flush()

    def spinner_task():
        while is_running:
            render_spinner()
            time.sleep(0.1)

    try:
        spinner_thread = threading.Thread(target=spinner_task)
        spinner_thread.start()
        yield
    finally:
        is_running = False
        spinner_thread.join()
        # Clean up task data
        with lock:
            process_runindicator.tasks.pop(task_id, None)
            if parent_id and task_id in process_runindicator.hierarchy.get(parent_id, []):
                process_runindicator.hierarchy[parent_id].remove(task_id)
            # Render remaining tasks
            render_spinner()


if __name__ == "__main__":
    import time

    with process_runindicator(desc="Parent Process", task_id="parent") as parent:
        with process_runindicator(desc="Child Process 1", task_id="child1", parent_id="parent"):
            time.sleep(2)  # Simulate child process 1

        with process_runindicator(desc="Child Process 2", task_id="child2", parent_id="parent"):
            time.sleep(3)  # Simulate child process 2


Parent Process [0:00:00]: |
Parent Process [0:00:00]: |
   Child Process 1 [0:00:00]: /
Parent Process [0:00:00]: /
   Child Process 1 [0:00:00]: -
Parent Process [0:00:00]: -
   Child Process 1 [0:00:00]: \
Parent Process [0:00:00]: \
   Child Process 1 [0:00:00]: |
Parent Process [0:00:00]: |
   Child Process 1 [0:00:00]: /
Parent Process [0:00:00]: /
   Child Process 1 [0:00:00]: -
Parent Process [0:00:00]: -
   Child Process 1 [0:00:00]: \
Parent Process [0:00:00]: \
   Child Process 1 [0:00:00]: |
Parent Process [0:00:00]: |
   Child Process 1 [0:00:00]: /
Parent Process [0:00:00]: /
   Child Process 1 [0:00:00]: -
Parent Process [0:00:00]: -
   Child Process 1 [0:00:00]: \
Parent Process [0:00:00]: \
   Child Process 1 [0:00:00]: |
Parent Process [0:00:00]: |
   Child Process 1 [0:00:00]: /
Parent Process [0:00:00]: /
   Child Process 1 [0:00:00]: -
Parent Process [0:00:00]: -
   Child Process 1 [0:00:00]: \
Parent Process [0:00:00]: \
   Child Process 1 [0:00:00]: |
Parent Proce